In [1]:
import pandas as pd
import pickle 

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import root_mean_squared_error as rmse

In [2]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']]
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,trip_distance
0,2023-01-01 00:32:10,2023-01-01 00:40:36,161,141,0.97
1,2023-01-01 00:55:08,2023-01-01 01:01:27,43,237,1.10
2,2023-01-01 00:25:04,2023-01-01 00:37:49,48,238,2.51
3,2023-01-01 00:03:48,2023-01-01 00:13:25,138,7,1.90
4,2023-01-01 00:10:29,2023-01-01 00:21:19,107,79,1.43


In [3]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,trip_distance,duration
0,2023-01-01 00:32:10,2023-01-01 00:40:36,161,141,0.97,8.433333
1,2023-01-01 00:55:08,2023-01-01 01:01:27,43,237,1.10,6.316667
2,2023-01-01 00:25:04,2023-01-01 00:37:49,48,238,2.51,12.750000
3,2023-01-01 00:03:48,2023-01-01 00:13:25,138,7,1.90,9.616667
4,2023-01-01 00:10:29,2023-01-01 00:21:19,107,79,1.43,10.833333


In [4]:
df.describe()

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,trip_distance,duration
count,3066766,3066766,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06
mean,2023-01-17 00:22:26.288164,2023-01-17 00:38:06.427874,1.663980e+02,1.643926e+02,3.847342e+00,1.566900e+01
min,2008-12-31 23:01:42,2009-01-01 14:29:11,1.000000e+00,1.000000e+00,0.000000e+00,-2.920000e+01
25%,2023-01-09 16:21:57.250000,2023-01-09 16:37:06,1.320000e+02,1.140000e+02,1.060000e+00,7.116667e+00
50%,2023-01-17 08:42:29.500000,2023-01-17 08:58:30.500000,1.620000e+02,1.620000e+02,1.800000e+00,1.151667e+01
75%,2023-01-24 16:26:27,2023-01-24 16:42:49,2.340000e+02,2.340000e+02,3.330000e+00,1.830000e+01
max,2023-02-01 00:56:53,2023-02-02 09:28:47,2.650000e+02,2.650000e+02,2.589281e+05,1.002918e+04
std,NaN,NaN,6.424413e+01,6.994368e+01,2.495838e+02,4.259435e+01


In [5]:
new_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
new_df.shape

(3009173, 6)

In [6]:
percentage = new_df.shape[0] / df.shape[0] * 100
print(percentage)

98.1220282212598


In [7]:
categorical = ['PULocationID', 'DOLocationID']
numerical =['trip_distance']

new_df[categorical] = new_df[categorical].astype(str)

/tmp/ipykernel_25999/1288593570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[categorical] = new_df[categorical].astype(str)


In [8]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']]
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [10]:
train_dicts = new_df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = new_df[target].values

In [11]:
lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
print("RMSE:", rmse(y_train, y_pred))

RMSE: 7.658396898909143


In [12]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_val[target].values

In [13]:
y_pred = lr.predict(X_val)
rmse(y_val, y_pred)

7.820263388747155